In [5]:
!git clone https://github.com/Francesco9932/mom_rag

Cloning into 'mom_rag'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 42 (delta 19), reused 15 (delta 3), pack-reused 0
Receiving objects: 100% (42/42), 1.83 MiB | 6.16 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [7]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q -U bitsandbytes
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu

!pip install accelerate
!pip install -q pypdf
!pip install -q spacy
!python3 -m spacy download it_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 40.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from langchain import HuggingFacePipeline
from langchain import PromptTemplate
from langchain.chains import RetrievalQA

import os

In [8]:
pdf_folder_path = "./mom_rag/bandi/"
documents = []
for file in os.listdir(pdf_folder_path):
    if file.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder_path, file)
        loader = PyPDFLoader(pdf_path)
        documents.extend(loader.load())

In [9]:
print(documents[0].page_content)

 
 
 
Data dipubblicazione: 29/08/2023
Nome allegato: Allegato 1-Capitolato Tecnico.pdf
CIG: A004C61876;
Nome procedura: Fornitura disoftware digrafica vettoriale e
software dimodellazione parametrica perilBIMperleesigenze
dell'Area tecnico edilizia.


In [10]:
splitter = RecursiveCharacterTextSplitter()
docs = splitter.split_documents(documents)

In [11]:
docs[2]

Document(page_content='2  \n1 PREMESSA  ................................ ................................ ................................ ..............................  3 \n2 CARATTERISTICHE DELLA FORNITURA  ................................ ................................ .................  4 \n2.1 OGGETTO DELLA FORNITURA  ................................ ................................ .............................  4 \n2.2 DURATA DELLA FORNITURA  ................................ ................................ ...............................  4 \n3 DESCRIZIONE DETTAGLIATA DELLA FORNITURA ................................ ................................ .. 5 \n3.1 SOTTOSCRIZIONE DI NUOVE LICENZE SOFTWARE AUTODESK  ................................ .........  5 \n4 CONSEGNA, INSTALLAZIONE E COLLAUDO DEL SOFTWARE  ................................ .................  6 \n5 LIVELLI DI SERVIZIO E PENALI  ................................ ................................ .................

In [12]:
# Define the path to the pre-trained model you want to use
modelPath = "nickprock/sentence-bert-base-italian-uncased"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
# use cpu or gpu
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_kwargs = {'device': device}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/118 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/243k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/732k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
db = FAISS.from_documents(docs, embeddings)

In [14]:
MODEL_NAME = "galatolo/cerbero-7b"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# Senza quantizzazione servono 15GB di VRAM

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
    # offload_folder="offload"
    quantization_config=quantization_config
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(
    pipeline=pipeline,
    )


tokenizer_config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

...

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/9.08G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [26]:
prompt_template = """
Sei un esperto di bandi e concorsi pubblici. Hai memorizzato tutti i bandi mai creati.
Utilizza le regole nella tua memoria insieme a qualsiasi altro contesto che possiedi per rispondere alle domande sul bando.


{context}


[|Umano|] {question}
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=False,
    chain_type_kwargs={"prompt": prompt},
)

In [32]:
input = "Devo creare un nuovo bando di Sofware CRM Salesforce, crea un capitolo oggetto della fornitura e riempilo con le informazioni in tuo possesso dei bandi ma per questo software"
result_ = chain(
    input
)
result = result_["result"].strip()

# print(input)
print(result)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Sei un esperto di bandi e concorsi pubblici. Hai memorizzato tutti i bandi mai creati.
Utilizza le regole nella tua memoria insieme a qualsiasi altro contesto che possiedi per rispondere alle domande sul bando.


6 4 Consegna, installazione e collaudo del software   
I programmi software oggetto di fornitura devono essere consegnati all’Istituto 
entro il termine di due giorni solari decorrenti dalla data di avvio dell’esecuzione 
del Contratto.  
La consegna dei programmi software prevista dal presente contratto, la rela tiva 
documentazione, nonché tutti gli aggiornamenti di release dei prodotti dovrà 
essere effettuata tramite la messa a disposizione per il download elettronico 
attraverso il sito web del produttore. Accedendo a detto sito, sarà possibile 
scaricare i p rogra mmi software e la relativa documentazione nella versione  
disponibile per tutta la durata del pre sente contratto per ognuno dei p rogrammi 
software oggetto della fornitura.  
La messa a disposizione sul sito